In [1]:
#Importing necessary models
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
#loading our mistral model
llm = Ollama(model="gemma3:4b")

/var/folders/hv/3fk9wmx12gs74mp51cnpmhw80000gq/T/ipykernel_11061/3995383221.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:4b")


In [3]:
filepath = ["/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/Robert Greene - The 48 Laws Of Power-Viking Penguin Group (2000).pdf"]

In [4]:
async def read_pdfs_into_pages(filepaths):
    pages = []
    for filepath in filepaths:
        loader = PyPDFLoader(filepath)
        async for page in loader.alazy_load():
            pages.append(page)
    return pages

In [5]:
def split_pages(pages):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
    )

    docs = text_splitter.split_documents(pages)
    return docs

In [6]:
def create_faiss_embeddings(docs):
    #All have been splitted correctly, now time to load embeddings creator
    embedding = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
    )   
    embeddings = FAISS.from_documents(docs, embedding)
    return embeddings

In [7]:
def get_answers_from_FAISS_usingllm(FAISS_embeddings, query):
    qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=FAISS_embeddings.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

    return(qa_chain.invoke(query))

In [10]:
book = await read_pdfs_into_pages(filepath)

In [12]:
book_split = split_pages(book)

In [13]:
book_embeddings = create_faiss_embeddings(book_split)

/var/folders/hv/3fk9wmx12gs74mp51cnpmhw80000gq/T/ipykernel_11061/2915709159.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/marketingvm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
question = """
How many chapters are there int he book 48 laws of power? what are the names of the chapters?
"""
response = get_answers_from_FAISS_usingllm(book_embeddings, question)
print(response)

{'query': '\nHow many chapters are there int he book 48 laws of power? what are the names of the chapters?\n', 'result': 'The text doesn’t state the number of chapters or their names. It only mentions “The 48 Laws of Power.”', 'source_documents': [Document(id='018c4f6c-58ba-4669-9a68-856d2f6e0b47', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': '/Users/hitesh.modi/Desktop/Kinda Personal/LLM Marketing Bot/Robert Greene - The 48 Laws Of Power-Viking Penguin Group (2000).pdf', 'total_pages': 476, 'page': 22, 'page_label': '23'}, page_content='the 48 laws in the table of contents, you can identify the pertinent law.\nFinally, the book can be browsed through and picked apart for enter-\ntainrnent. for an enjoyable ride through the foibles and great deeds of our\npredecessors in power. A warning, however, to those who use the book for\nthis purpose: It might be better to tum back. Power is endlessly seductive\nand deceptive in its own way. It is a 1abyrinth—